

![Egeria Logo](https://raw.githubusercontent.com/odpi/egeria/main/assets/img/ODPi_Egeria_Logo_color.png)

### Egeria Workbook

# Receiving Open Lineage Events via Kafka

## Introduction

Open Lineage supports a proxy backend server that is able to distribute events via Apache Kafka.  This workbook sets up Egeria to receive these events.

---

In [1]:
# Initialize pyegeria

%run ../../pyegeria/initialize-pyegeria.ipynb


In [2]:
# Create client to egeria

egeria_tech = EgeriaTech(view_server, url, user_id, user_pwd)
token = egeria_tech.create_egeria_bearer_token()

----

There is a governance action process called *HarvestOpenLineageEvents:CreateAsCatalogTargetGovernanceActionProcess* that sets up Apache 

In [3]:
createAndCatalogServerName="HarvestOpenLineageEvents:CreateAsCatalogTargetGovernanceActionProcess"

process_guid = egeria_tech.get_element_guid_by_unique_name(createAndCatalogServerName)

process_graph = egeria_tech.get_gov_action_process_graph(process_guid)
print_governance_action_process_graph(process_graph)


-----

We begin by defining which Apache Kafka topic is being used by other systems to publish open lineage events, and then call the governance action process.

----

In [4]:

requestParameters = {
    "serverName" : "LocalKafka2",
    "hostIdentifier": "localhost",
    "portNumber": "9092",
    "description" : "Receive open lineage events via Kafka Topic.",
    "topicDescription": "Open Lineage Events for Backend Proxy",
    "versionIdentifier" : "V1.0",
    "fullTopicName": "openlineage.events",
    "shortTopicName": "openlineage.events",
    "eventDirection": "inOut"
}

egeria_tech.initiate_gov_action_process(createAndCatalogServerName, None, None, None, requestParameters, None, None)



'dbfebf22-a6a4-4a4e-9998-e2aefd41c76e'

----

The command below displays the latest governance actions.  You should see they are in **ACTIONED** status.  If you see failures it means that either Apache Kafka is not running or the values describing its location are not correct.  If Apache Kafka is down, restart it and re-run the cell above.  If you realize one or more of the values describing topic to listen on is not right, correct the values and retry the process.

----

In [5]:

display_engine_activity_c(row_limit=2)


                                                     Engine Action Status for Platform https://host.docker.internal:9443 @ Tue Mar 18 17:31:22 2025                                                     
╭─────────────────────┬─────────────────────────────────────────────┬─────────────────────────────────────────────┬───────────────┬─────────────────────┬──────────────────────────────────────────────╮
│ Requested Time      │ Core Info                                   │ Target Elements                             │ Action Status │ Completion Time     │ Core Results                                 │
├─────────────────────┼─────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────────┼─────────────────────┼──────────────────────────────────────────────┤
│ 2025-03-18T17:31:16 │                                             │                                             │ ACTIONED      │ 2025-03-18T17:31:19 │                                           

----

The effect of the process is to configure the *OpenLineageKafkaListener* connector to start listening for Open Lineage Events.
The Target Element column shows the details of the Apache Kafka Topics it is listening to.

----

In [ ]:

display_integration_daemon_status(['OpenLineageKafkaListener'], paging=True, width=200)
